# Patches from center only: HE
Training set: patches from Y-pos vs Y-neg images.  
No attempt to distinguish background, stroma, tumor.  
Populate training subdirectories named Ypos or Yneg.  

In [1]:
import time
import os
import glob
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import numpy as np
import json
import tensorflow as tf
from tensorflow import keras
import keras.layers as kl
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
DIR_IMAGES_RAW = "/home/jrm/Martinez/images/raw/"
DIR_IMAGES_TRAIN = "/home/jrm/Martinez/images/HEcenterpatch/train/"
DIR_IMAGES_VALID = "/home/jrm/Martinez/images/HEcenterpatch/valid/"
PATTERN_IMAGES_RAW = "*.HE1.*.tif"
PATCH_SIZE=224  # matches VGG
IMAGE_SIZE = (PATCH_SIZE,PATCH_SIZE)
DF_Ypos = ['B7.','B15','D1.','D5.','E7.','E9.','F9.','G3.','H13','I1.','I5.','I13']
DF_Yneg = ['A3.','A5.','B13','C1.','C11','D3.','E5.','F3.','F7','F11','F13','F15','G15','H1.','H3.','H7.','H15']

In [3]:
def get_image_names(path,pattern):
    paths = glob.glob(path+pattern)
    names = [os.path.basename(x) for x in paths]
    return names
FILENAMES_IMAGES_RAW = get_image_names(DIR_IMAGES_RAW,PATTERN_IMAGES_RAW)

In [9]:
class patch_maker:
    def __init__(self):
        self.path=""
        self.w=0
        self.h=0
        self.im_width = 0
        self.im_height = 0
        self.patch_size = 10 # scalar, assumed square for now
        self.pixel_array = None
        self.output_dir = ""
        self.output_format='.jpg'
        self.center=(0.0,0.0)
        self.radial_threshold = 0.0
        self.BORDER = 200
    def set_input_path(self,path):
        self.path=path
    def set_output_path(self,path):
        self.output_dir = path
    def set_patch_size(self,scalar):
        self.patch_size = scalar
    def load_pixel_array(self,filename,verbose=False):
        self.w=0
        self.h=0
        im = Image.open(self.path+filename)
        ima = np.array(im)   # convert to numpy
        self.im_width = ima.shape[1]
        self.im_height = ima.shape[0]
        self.center=(self.im_width/2,self.im_height/2)
        self.radial_threshold = min(self.im_width,self.im_height)/2-self.BORDER
        if verbose:
            print(filename, ima.size, ima.shape)
        self.pixel_array = ima
        return ima
    def radius(self,x,y):
        return np.sqrt((x-self.center[0])**2+(y-self.center[1])**2)
    def get_next_patch(self): 
        IM_WIDTH = self.im_width
        IM_HEIGHT = self.im_height
        PIXEL_ARRAY = self.pixel_array
        patch = None
        if self.w+self.patch_size > IM_WIDTH:
            self.h += self.patch_size
            self.w = 0
        if self.w+self.patch_size <= IM_WIDTH and self.h+self.patch_size <= IM_HEIGHT:
            patch = PIXEL_ARRAY[self.h:self.h+self.patch_size, self.w:self.w+self.patch_size]
            self.w += self.patch_size
        return patch
    def get_next_center_patch(self):
        patch = self.get_next_patch()
        while (patch is not None):
            radius1 = self.radius(self.w,self.h)
            radius2 = self.radius(self.w,self.h+self.patch_size)
            radius3 = self.radius(self.w+self.patch_size,self.h)
            radius4 = self.radius(self.w+self.patch_size,self.h+self.patch_size)
            max_radius=max(radius1,radius2,radius3,radius4)
            if max_radius < self.radial_threshold:
                #print("Good",self.w,self.h)
                return patch
            #print("Bad",self.w,self.h)
            patch = self.get_next_patch()
        return None
    def save_patch (self, prefix, pnum, patch):
        ext = self.output_format
        path= self.output_dir+prefix+'.'+str(pnum)+ext
        im =  Image.fromarray(patch)
        im.save(path)

In [10]:
pm = patch_maker()
pm.set_input_path(DIR_IMAGES_RAW)
pm.set_patch_size(PATCH_SIZE)
pm.set_output_path(DIR_IMAGES_TRAIN)  # later, move 20% of files to VALID
for filename in FILENAMES_IMAGES_RAW:
    print(filename,end=" ")
    file_counts = {'Yneg':0, 'Ypos':0}
    neg_or_pos = 0
    prefix = filename[0:3]
    if prefix[2] == '.':  # we have prefixes like F1. and F13
        prefix=prefix[0:2]+'_'   # avoid the extra period
    neg_or_pos = +1
    phenotype = 'Ypos'
    if prefix in DF_Yneg:
        neg_or_pos = -1
        phenotype = 'Yneg'
    pixels = pm.load_pixel_array(filename)
    print(pixels.shape,end=" ")
    patch = pm.get_next_center_patch()
    while patch is not None:
        file_counts[phenotype] += 1
        pm.save_patch(phenotype+'/'+prefix,file_counts[phenotype],patch)
        patch = pm.get_next_center_patch()
    print(file_counts)

I1.HE1.09.tif (10439, 9446, 3) {'Yneg': 0, 'Ypos': 1202}
C1.HE1.01.tif (10641, 9851, 3) {'Yneg': 0, 'Ypos': 1313}
B7.HE1.60.tif (10236, 10034, 3) {'Yneg': 0, 'Ypos': 1363}
D5.HE1.43.tif (10256, 10054, 3) {'Yneg': 0, 'Ypos': 1370}
H15.HE1.132.tif (10641, 9851, 3) {'Yneg': 1313, 'Ypos': 0}
G3.HE1.25.tif (10662, 9223, 3) {'Yneg': 0, 'Ypos': 1143}
F15.HE1.135.tif (10236, 10257, 3) {'Yneg': 1421, 'Ypos': 0}
E9.HE1.75.tif (10844, 10236, 3) {'Yneg': 0, 'Ypos': 1425}
H13.HE1.114.tif (11047, 9831, 3) {'Yneg': 0, 'Ypos': 1309}
F9.HE1.80.tif (10844, 10439, 3) {'Yneg': 0, 'Ypos': 1489}
I5.HE1.48.tif (10439, 10459, 3) {'Yneg': 0, 'Ypos': 1491}
B15.HE1.134.tif (10236, 10054, 3) {'Yneg': 0, 'Ypos': 1371}
D1.HE1.02.tif (10459, 10257, 3) {'Yneg': 0, 'Ypos': 1434}
B13.HE1.115.tif (10641, 10439, 3) {'Yneg': 1488, 'Ypos': 0}
D3.HE1.20.tif (10256, 10034, 3) {'Yneg': 0, 'Ypos': 1366}
F3.HE1.22.tif (10641, 10034, 3) {'Yneg': 0, 'Ypos': 1367}
I13.HE1.117.tif (10439, 10236, 3) {'Yneg': 0, 'Ypos': 1431}
G15.HE1

In [11]:
# Move a portion of the train files into the valid directory.
from random import sample
VALIDATION_PORTION = 0.20
for phenotype in ('Ypos','Yneg'):
    fromdir = DIR_IMAGES_TRAIN+"/"+phenotype+"/"
    todir   = DIR_IMAGES_VALID+"/"+phenotype+"/"
    files = os.listdir(fromdir)
    total = len(files)
    selected = sample(files, int(VALIDATION_PORTION*total))
    for vfile in selected:
        os.rename(fromdir+vfile,todir+vfile)
print("Done")

Done
